In [1]:
#Create spark session
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName("CovidProject").getOrCreate()
spark


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/08 00:55:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#Read Parquet file in a dataframe
hospital_df = spark.read.parquet('hospital_data/hospital_data.parquet')
hospital_df.printSchema()
hospital_df.show(10)

+--------------------+--------------+--------------------+-----------------------+
|       Hospital_Name|Beds_Available|COVID_Beds_Available|NonCOVID_Beds_Available|
+--------------------+--------------+--------------------+-----------------------+
|    Pune Hospital 87|            66|                  58|                      8|
|    Pune Hospital 44|           186|                  79|                    107|
|    Pune Hospital 84|           111|                  23|                     88|
|    Pune Hospital 11|           124|                  35|                     89|
|Hyderabad Hospita...|           199|                  30|                    169|
|  Mumbai Hospital 62|           183|                 113|                     70|
|  Mumbai Hospital 64|            65|                  28|                     37|
|   Delhi Hospital 60|           140|                  33|                    107|
|   Delhi Hospital 15|           105|                  72|                     33|
|Bhu

In [9]:
#Read avro file in a dataframe
patient_df = spark.read.format("avro").load('/Users/meghasinghrajpoot/Documents/GitHub/DataEngineeringProject/covid_patient_data.avro')
patient_df.printSchema()
patient_df.show(10)

SyntaxError: invalid syntax (2984300799.py, line 2)